# Evaluation of different crowd analysis metrics


(1) Dataset filtering:

* Exclude workers based on:
    - contradiction ration (absolut thresh, batch stdv, pair stdv, total stdv)
    - worker quality score (thresholds)
    - attention check fails 
    
(2) Label aggregation:

* Majority vote
* Top vote
* CT unit-label score (tresholds)

In [88]:
from load_data import load_experiment_data, load_gold_data
from evaluation import evaluate_configs

import pandas as pd

run = 4
group = '*'
n_q = '*'
batch = '*'
gold = load_gold_data(run, group, n_q, batch)
print('number of gold instances: ', len(gold))

# load crowd:
run = '*'
group = 'experiment*'
n_q = '*'
batch = '*'
crowd = load_experiment_data(run, group, n_q, batch)

overview_dicts = evaluate_configs(gold, crowd)
df =  pd.DataFrame(overview_dicts) 
df.sort_values(by=['relations-f1'], ascending=False)[['config',
                                                      'relations-f1',
                                                      'levels-f1', 'negative_relations-f1']]

number of gold instances:  63
Discarded 655.0 annotations.
63 17917 770
aggretation
cleaning and aggregation


,config,relations-f1,levels-f1,negative_relations-f1
28,"(batch, 0.5, contradictions, majority_vote)",0.895726,0.930660,0.874621
45,"(total, 0.5, contradictions, top_vote)",0.887392,0.861320,0.866315
13,"(pair, 0.5, contradictions, top_vote)",0.873016,0.861320,0.849057
53,"(total, 1.5, contradictions, top_vote)",0.850693,0.861320,0.823145
49,"(total, 1, contradictions, top_vote)",0.850693,0.861320,0.823145
16,"(pair, 1, contradictions, majority_vote)",0.850099,0.828287,0.820107
44,"(total, 0.5, contradictions, majority_vote)",0.850099,0.861320,0.820107
52,"(total, 1.5, contradictions, majority_vote)",0.850099,0.861320,0.820107
48,"(total, 1, contradictions, majority_vote)",0.850099,0.861320,0.820107
12,"(pair, 0.5, contradictions, majority_vote)",0.847813,0.861320,0.818014


In [3]:
#print(df.to_latex(index=False))

# Distinguishing ambiguous/difficult instances from low quality annotations


* detect wrong potentially wrong or problematic answers





In [89]:
from load_data import load_experiment_data, load_gold_data
from clean_annotations import clean_workers
from evaluation import evaluate_configs

import pandas as pd

from load_data import load_experiment_data
from calculate_iaa import get_agreement
#from utils_analysis import load_contradiction_pairs
#from utils_analysis import collect_contradictions
from utils_analysis import sort_by_key
#from utils_analysis import get_annotation_ids
from utils_analysis import load_analysis
from utils_analysis import load_ct
from evaluation import get_evaluation_instances
from aggregation import aggregate_binary_labels
from clean_annotations import remove_contradicting_workers
import statistics

#from collections import Counter, defaultdict
#import pandas as pd
#from scipy.stats import spearmanr

run = "*"
group = 'experiment*'
batch = '*'
n_q = '*'

crowd = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
crowd_eval = get_evaluation_instances(crowd, gold)

crowd_eval = clean_workers(crowd_eval, run, group,
                                               batch, 'contradictions', 'total', 1)

run = 4
group = '*'
n_q = '*'
batch = '*'
gold = load_gold_data(run, group, n_q, batch)



Discarded 655.0 annotations.
63 17917 770


In [131]:
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support as p_r_f1

run = "*"
group = 'experiment*'
batch = '*'
n_q = '*'

analysis_type = 'workers'
worker_analysis =  load_analysis(analysis_type, run, group, batch, as_dict=True)

analysis_type = 'units'
ct_dicts_units = load_ct(run, group, batch, analysis_type, as_dict=True)
analysis_type = 'workers'
ct_dicts_workers = load_ct(run, group, batch, analysis_type, as_dict=True)
#ag_cont_dicts = get_agreement_contradiction_data(data_dict_list, pair_analysis, ct_dicts)
#df_ag_cont = pd.DataFrame(ag_cont_dicts)

crowd_eval_agg = aggregate_binary_labels(crowd_eval, ct_dicts_units)

analysis_by_worker = sort_by_key(worker_analysis, ['workerid'])

ct_by_worker = sort_by_key(ct_dicts_workers, ['worker'])
ct_by_unit = sort_by_key(ct_dicts_units, ['unit'])
answers_by_quid = sort_by_key(crowd_eval_agg, ['relation', 'property', 'concept'])

test_run = ['roll', 'red', 'round']
data_by_triple = sort_by_key(crowd_eval, ['relation', 'property', 'concept'])
gold_by_quid = sort_by_key(gold, ['quid'])
gold_by_triple = sort_by_key(gold, ['relation', 'property', 'concept'])

run_dict = defaultdict(list)
answers = []
predictions = []

for triple, data in data_by_triple.items():
    quid = data[0]['quid']
    cont_sum = 0.0
    cont_list = []
    wqs_sum = 0.0
    wqs_list = []
    n_workers = len(data)
    uqs = ct_by_unit[quid][0]['uqs']
    #print(triple, quid)
    answer = str(answers_by_quid[triple][0]['majority_vote']).lower()
    triple = f"{data[0]['relation']}-{data[0]['property']}-{data[0]['concept']}"
    prop = data[0]['property']
    if prop in test_run:
        test = True
    else:
        test = False
    answer_gold = str(gold_by_triple[triple][0]['answer']).lower()
    iaa = get_agreement(data, v=False, disable_kappa=True)
    if answer == answer_gold:
        correct = True
    else:
        correct = False
    answers.append(correct)
    run_dict[test].append(correct)
    for d in data:
        worker = d['workerid']
        worker_analysis_dict = analysis_by_worker[worker][0]
        cont_rate = worker_analysis_dict['contradiction_poss_contradiction_ratio']
        cont_sum += cont_rate
        cont_list.append(cont_rate)
        wqs_sum += ct_by_worker[worker][0]['wqs']
        wqs_list.append(ct_by_worker[worker][0]['wqs'])
        
    cont_med = statistics.median(cont_list)
    cont_mean = cont_sum / len(cont_list)
    wqs_med = statistics.median(wqs_list)
    
    if  uqs > 0.50: #and correct == True:
        predictions.append(True)
        if correct == True:
            print(triple, wqs_med, answer, answer_gold)
        pass
        #predictions.append(True)
        #print('correct answer identified', triple, correct)
    
        #print('identified:', triple, answer)
    elif uqs <= 0.50: # and correct == False:
        predictions.append(False)
        if correct == False:
            print('incorrect answer identified', triple, correct)
    

        #predictions.append(False)
        #print('risky unit identified:', triple, answer, correct)
  
        #print(triple, correct, cont_med)
p, r, f1, support = p_r_f1(answers, predictions, average = 'weighted') 
print(f'precision {p}, recall {r}, f1 {f1} ')

typical_of_concept-fly-arrow 0.6796334308426818 true true
impossible-fly-arrow 0.6796334308426818 false false
rare-roll-pin 0.6148316539446073 false false
unusual-roll-pin 0.6750428924666586 false false
impossible-roll-pin 0.6750428924666586 false false
incorrect answer identified implied_category-yellow-buttercup False
typical_of_concept-yellow-buttercup 0.7311694232474322 true true
implied_category-roll-tire 0.7551381325387114 true true
typical_of_property-roll-tire 0.7551381325387114 true true
typical_of_concept-roll-tire 0.7551381325387114 true true
afforded_unusual-roll-tire 0.7551381325387114 false false
afforded_usual-roll-tire 0.7551381325387114 true true
variability_limited-roll-tire 0.7551381325387114 false false
creative-roll-tire 0.7551381325387114 false false
rare-roll-tire 0.7551381325387114 false false
impossible-roll-tire 0.7551381325387114 false false
implied_category-roll-shovel 0.6781881476242915 false false
incorrect answer identified afforded_unusual-roll-shovel Fa

7 16
test run False: 0.4375
21 25
test run True: 0.84
